# Conformer generator

In [1]:
import veloxchem as vlx

We define a structure by using a SMILES string.

In [2]:
molecule = vlx.Molecule.read_smiles(
    "CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C"
)  # this is SMILES for Penicillin G
molecule.show(atom_indices=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

ConformerGenerator class can generate all possible conformations and apply energy minimization process with MMforcefield

In [3]:
conf = vlx.ConformerGenerator()
conformers_dict = conf.generate(molecule)

***********                                                                                                               
* Warning * ConformerGenerator: Partial charges not provided. Will use a quick (and likely inaccurate) estimation of partial charges.
***********                                                                                                               
* Info * 36 conformers will be generated.                                                                                 
* Info * 36 conformers generated in 0.34 sec                                                                              
* Info * Energy minimization of 36 conformers took 1.24 sec                                                               
* Info * Global minimum energy: -39.074 kJ/mol                                                                            
* Info * 14 conformers remain after removal of duplicate conformers.                                                      
* Inf

show the lowest energy conformer

In [4]:
conf.show_global_minimum()

Global minimum conformer with energy -39.074 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

 show more conformers

In [5]:
conf.show_conformers(number=3, atom_indices=True)

Conformer 1 with energy -39.074 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Conformer 2 with energy -37.305 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Conformer 3 with energy -36.317 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

we can provide resp charges as partial charges to improve the used MMforcefield quality

In [6]:
basis = vlx.MolecularBasis.read(molecule, "6-31g*")
resp = vlx.RespChargesDriver()
resp.ostream.mute()
partial_charges = resp.compute(molecule, basis, 'resp')

In [7]:
conf = vlx.ConformerGenerator()
conf.partial_charges = partial_charges
conformers_dict = conf.generate(molecule)
conf.show_global_minimum(atom_indices=True)

* Info * 36 conformers will be generated.                                                                                 
* Info * 36 conformers generated in 0.59 sec                                                                              
* Info * Energy minimization of 36 conformers took 1.68 sec                                                               
* Info * Global minimum energy: -174.134 kJ/mol                                                                           
* Info * 8 conformers remain after removal of duplicate conformers.                                                       
* Info * Total time spent in generating conformers: 2.56 sec                                                              
Global minimum conformer with energy -174.134 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

we can apply implicit solvent model for the energy minimization 

In [8]:
conf = vlx.ConformerGenerator()
conf.ostream.mute()
conf.show_available_implicit_solvent_models()
conf.implicit_solvent_model = "hct"  #here we set Hawkins-Cramer-Truhlar GBSA model
conformers_dict = conf.generate(molecule)
conf.show_global_minimum(atom_indices=True)

Available implicit solvent files:
gbn2.xml
gbn.xml
obc1.xml
hct.xml
obc2.xml
Global minimum conformer with energy -136.189 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

show more conformers

In [9]:
conf.show_conformers(number=3, atom_indices=True)

Conformer 1 with energy -136.189 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Conformer 2 with energy -133.242 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Conformer 3 with energy -129.934 kJ/mol


3Dmol.js failed to load for some reason. Please check your browser console for error messages.